<a href="https://colab.research.google.com/github/klipbn/klip_alex/blob/main/determination_average_number_lodsman_users/determination_average_number_lodsman_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Key ID:  
458744584072543631 - not actived  
719635943846940421 - ЛОЦМАН:КБ 2022,HSL,сетвр  
834732166184233058 - ОЭ КОМПАC-3D v20,HSL,сетвр  
982786152807025115 - ЛОЦМАН:КБ 2021,HSL,сетвр  

Productid:  
3197 ЛОЦМАН:КБ 2022,HSL,сетвр   
3258 КОМПАC-3D v20,HSL,сетвр 

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_colwidth = 100

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Читаем данные с логами:

In [20]:
for i in range(1, 15):

  # чтение файла
  df = pd.read_csv(f"/content/drive/My Drive/Projects/determination_average_number_lodsman_users/logs/2021_11_{i:02d}_access.log", 
                  sep=r' ', skiprows=1, error_bad_lines=False, warn_bad_lines=False) 
  # задаем признаки
  df.columns = ['nan',	'nan1',	'date', 'time',	'ip',	'user',	'port',	'api' ,	'features' ,	'result' ,	'nan2']
  # уберем лишнии признаки
  df.drop(['nan', 'nan1', 'nan2'], axis=1, inplace=True)
  # удалим пропущенные значения
  df = df.dropna(subset=['user', 'features'])

  # проведем логическую индексацию
  mask = df["user"].str.contains(r"Администратор")
  dff = df.loc[mask]
  mask = dff["features"].str.contains(r"productid=3197")
  dff = dff.loc[mask]


# соединим логи в одну таблицу
  if i == 1: 
    df_null = pd.DataFrame([np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]).T
    df_null.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]

    df_null = pd.concat([df_null, dff], ignore_index=True)
  else:

    df_null = pd.concat([df_null, dff], ignore_index=True)  

# уберем лишние признаки и пропущенные значения из созданного переходного датафрейма
df_null.drop(['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype'], axis=1, inplace=True) 
df_null = df_null.dropna()

Сделаем из признака `'features'` датафрейм:

In [21]:
for i in range(len(df_null)-1):
  if i == 0: 
    zz555 = pd.DataFrame([np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]).T
    zz555.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
    zz = pd.DataFrame(str(df_null["features"][i:i+1]).split(',')).T
    zz.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
    zz2 = pd.DataFrame(str(df_null["features"][i+1:i+2]).split(',')).T
    zz2.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
    zz555 = pd.concat([zz555, zz, zz2], ignore_index=True)
  else:
    zz = pd.DataFrame(str(df_null["features"][i:i+1]).split(',')).T
    zz.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
    zz2 = pd.DataFrame(str(df_null["features"][i+1:i+2]).split(',')).T
    zz2.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
    zz555 = pd.concat([zz555, zz, zz2], ignore_index=True)

zzz5556 = zz555.drop_duplicates().dropna().reset_index()   # датафрейм из признака 

Соединим наши датафреймы с признаком `'features'` и с логами. Уберем лишние признаки, пропуски:

In [22]:
df = df_null.merge(zzz5556, how='outer', left_index=True, right_index=True)
df.drop(['port', 'features', 'result', 'index', 'median_time', 'span', 'dtype'], axis=1, inplace=True)
df = df.dropna().reset_index()

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3109 entries, 0 to 3108
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      3109 non-null   int64 
 1   date       3109 non-null   object
 2   time       3109 non-null   object
 3   ip         3109 non-null   object
 4   user       3109 non-null   object
 5   haspid     3109 non-null   object
 6   productid  3109 non-null   object
 7   feat       3109 non-null   object
 8   sess       3109 non-null   object
 9   api        3109 non-null   object
dtypes: int64(1), object(9)
memory usage: 243.0+ KB
